In [1]:
# !python -m pip install --upgrade pip

In [2]:
# #INSTALAÇÃO DAS LIBS
# !pip install yfinance
# !pip install crewai
# !pip install 'crewai[tools]'
# !pip install langchain
# !pip install langchain-openai
# !pip install langchain-community
# !pip install duckduckgo-search

In [3]:
# Import das libs
import json
import os
from datetime import datetime

import yfinance as yf
from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI

from langchain_community.tools import DuckDuckGoSearchResults



c:\Users\jjako\Documents\programacao\ufes\rocket-seat\ia-na-pratica\env\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
# Criando Yahoo Finance Tool
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stock

yahoo_finance_tool = Tool(
  name="Yahoo finance Tool",
  description="fetches stocks price for {ticket} form the last year about a specific company from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)


In [5]:
# Importandoo OpenAi LLM - GPT
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
stockPriceAnalyst = Agent(
  role="Senior stock price Analyst",
  goal="Find the {ticket} stock price and analyses trends",
  backstory=""" You're highly experienced in analyzing the price of an specific stock
  and make predictions about its future price.
  """,
  verbose=True,
  llm=llm,
  max_iter=5,
  memory=True,
  tools=[yahoo_finance_tool],
  allow_delegation=False
)

In [7]:
getStockPrice = Task(
  description="Analyze the stock {ticket} price history and create a trend analysis of up, down or sideways",
  expected_output=""" Specify the current trend stock price - up, down, or sideways.
  eg. stock= 'APPL, price UP'
  """,
  agent=stockPriceAnalyst
)

In [8]:
# Importando a tool do search
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [9]:
newsAnalyst = Agent(
  role="Senior Stock News Analyst",
  goal="""Create a short summary of the market news related to the stock ticket 
  {ticket} company. Specify the current trend - up, down, sideways - with the news context.
  For each requested stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed""",
  backstory=""" You're highly experienced in analyzing the market trends and news and have tracked assets for more than 10 years.
  
  You're alsoo master level analyst in the traditional market and have deep understanding of human psychology
  
  You understand news, their titles and information, but you look at those with a health dose of skepticism.
  You consider also the source of the news article.
  """,
  verbose=True,
  llm=llm,
  max_iter=10,
  memory=True,
  tools=[search_tool],
  allow_delegation=False
)

In [10]:
getNews= Task(
  description=f"""Take the stock and always include BTC to it (if not requested).
  Use the search tool to search each one individualy.
  
  The current date is {datetime.now()}
  
  Compose the results into a helpful report.
  """,
  expected_output="""A summary of the overall market and one sentece summary for each request asset.
  Include a fear/greed score for each asset based on the news. Use the format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent=newsAnalyst,
)

In [11]:
stockAnalystWriter = Agent(
  role="Senior Stock Analyst Writer",
  goal="""Analyze the trends price and news and write an insightful compelling and informative 3 paragraph long newsletter based on the  stock report and price trend.""",
  backstory="""You're widely acceped as the best stock analyst in the market. You understand complex concepts and create compelling stories
  and narratives that resonate with wider audiences.
  
  You understand macro factors and combine multiple theories - e.g. cycle theory and fundamental analysis.
  You're able to hold multiple opinions when analysing anything.
  """,
  verbose=True,
  llm=llm,
  max_iter=5,
  memory=True,
  # tools=[search_tool]
  allow_delegation=True
)

In [12]:
writeAnalysis= Task(
  description="""Use the stock price trend and the stock news report to create an analysis and write the newsletter about the ticket {ticket} company
  that is brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
  Include the previous analysis of stock trend and news summary.
  """,
  expected_output="""An eloquent 3 paragraph newsletter formated as markdown in an easy readable manner. It should contains:
  - 3 bullets executive summary
  - Introduction - set the overall picture and spike up the interest
  - main part provides the meat of the analysis including the news summary and fear/greed scores
  - summary - key facts and concrete future trend prediction - up, down or sideways
  
  """,
  agent=stockAnalystWriter,
  context=[getStockPrice, getNews]
)

In [13]:
crew = Crew(
  agents=[stockPriceAnalyst, newsAnalyst, stockAnalystWriter],
  tasks=[getStockPrice, getNews, writeAnalysis],
  verbose=True,
  process=Process.hierarchical,
  full_output=True,
  share_crew=False,
  manager_llm=llm,
  max_iter=15
)

In [14]:
results = crew.kickoff(inputs={'ticket':"AAPL"})

 [2024-08-20 21:27:16][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 21:27:16][INFO]: == Starting Task: Analyze the stock AAPL price history and create a trend analysis of up, down or sideways


> Entering new CrewAgentExecutor chain...
I need to gather some information from the Senior Stock Price Analyst to ensure that we have a comprehensive analysis of AAPL's price history and trends. Since they have expertise in this area, I will delegate the task to them.

Action: Delegate work to coworker  
Action Input: {"task": "Analyze the stock AAPL price history and create a trend analysis of up, down or sideways", "context": "We need to analyze the historical stock price data of AAPL (Apple Inc.) to identify the current trend. The analysis should highlight whether the stock price is trending upwards, downwards, or is moving sideways based on the historical data. Please consider factors such as recent price movements, support and resistance levels, and any relevant market news that cou

[*********************100%%**********************]  1 of 1 completed


 

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643402   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043762   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884872   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546371   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date      

In [15]:
from IPython.display import Markdown, display

display(Markdown(results.raw[11:]))
# crew_output = results
# # Accessing the crew output
# print(f"Raw Output: {crew_output.raw}")
# if crew_output.json_dict:
#     print(f"JSON Output: {json.dumps(crew_output.json_dict, indent=2)}")
# if crew_output.pydantic:
#     print(f"Pydantic Output: {crew_output.pydantic}")
# print(f"Tasks Output: {crew_output.tasks_output}")
# print(f"Token Usage: {crew_output.token_usage}")


# AAPL Stock Newsletter: Navigating Mixed Sentiment and Future Prospects

**Executive Summary**  
- Apple Inc. (AAPL) is currently navigating a landscape of mixed market sentiment following Warren Buffett's Berkshire Hathaway's decision to divest a substantial portion of its holdings, totaling 389 million shares.  
- Despite these headwinds, AAPL's stock has demonstrated resilience, closing at approximately $226.57, and achieving a year-to-date gain of nearly 20%.  
- As analysts adjust their price targets upward, there are signs that the company is poised for long-term success, particularly with anticipated product launches.  

**Introduction**  
The recent sell-off by Berkshire Hathaway has sent ripples through the investment community, raising questions about the future trajectory of AAPL. However, the stock's ability to maintain a strong position amidst these developments speaks volumes about its underlying strength and market confidence. Currently trading within striking distance of its 52-week high and showing a solid year-to-date performance, AAPL is a focal point for investors balancing caution with optimism. The prevailing market sentiment, reflected in a Fear/Greed score of 45, indicates a cautious approach among investors who may be weighing the implications of significant sell-offs against the backdrop of upcoming product innovations and broader market dynamics.

**Detailed Analysis and Future Predictions**  
Apple's stock price trend illustrates a remarkable recovery from earlier lows this year, showcasing its resilience in the face of external pressures. While the recent sell-off by Berkshire Hathaway raises eyebrows, the stock's close proximity to its 52-week high suggests that many investors still view AAPL as a viable long-term investment. Analysts' upward revisions of price targets bolster this perspective, particularly as the company gears up for new product launches that could drive future revenue growth. However, the current Fear/Greed score of 45 reflects a prevailing sense of caution among investors, indicating that while the outlook is predominantly positive, the market remains vigilant. Moving forward, we can expect an upward trend, albeit with a cautious approach, as external sell-offs and macroeconomic factors continue to influence market sentiment. Overall, AAPL appears well-positioned for sustained growth, provided it can navigate the complexities of investor sentiment and deliver on its product promises.
```